In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from sklearn.metrics import roc_curve
from sklearn.metrics import fbeta_score as f2
from sklearn.metrics import f1_score as f1
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
datapath = "/data/cs3310/CordieLab/MSOE_Spreadsheet.xlsx"

In [2]:
import warnings
warnings.filterwarnings("ignore")

First, create dataframes of each spreadsheet separately. Then, merge both sheets on the "occurrence_no" column and set that as the index of the merged dataframe

In [3]:
raw_df = pd.read_excel(datapath, sheet_name=0, header=21)
#raw_df.set_index("occurrence_no")

main_df = pd.read_excel(datapath, sheet_name=1, header=0)
#main_df.set_index("occurrence_no")

merged_df = raw_df.merge(main_df, left_on="occurrence_no", right_on="occurrence_no")
merged_df.set_index("occurrence_no")

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25192 entries, 0 to 25191
Data columns (total 48 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   occurrence_no      25192 non-null  int64  
 1   record_type        25192 non-null  object 
 2   reid_no            109 non-null    float64
 3   flags              0 non-null      float64
 4   collection_no_x    25192 non-null  int64  
 5   accepted_name_x    25192 non-null  object 
 6   accepted_rank_x    25192 non-null  object 
 7   accepted_no_x      25192 non-null  int64  
 8   early_interval_x   25192 non-null  object 
 9   late_interval      1468 non-null   object 
 10  max_ma_x           25192 non-null  float64
 11  min_ma_x           25192 non-null  float64
 12  ref_author         25192 non-null  object 
 13  ref_pubyr          25192 non-null  int64  
 14  reference_no_x     25192 non-null  int64  
 15  phylum             25192 non-null  object 
 16  class              251

This is where the actual dataframe is made by selecting columns (by index number rather than name... God knows why I did this, feel free to change it). Do note that changing the columns in cleaned_df means that you may need to change the lists of categorical and numerical columns, since any columns you add will not be in the current list, and any columns you remove may cause an issue when it looks for a column name that isn't there

### Uncomment the cell below for Skeletal Material

In [6]:
#UNCOMMENT FOR SKELETAL MATERIAL
cleaned_df = merged_df.iloc[:, [0, 8, 10, 11, 15, 16, 17, 18, 19, 20, 28, 29, 30, 39, 45]]
cat_cols = ["early_interval_x", "phylum", "class", "order", "family", "genus", "cc", "geoplate", "formation", "Skeletal Material"]
cleaned_df["Skeletal Material"].replace("oragnic", "organic", inplace=True)
cleaned_df["Skeletal Material"].value_counts()

### Uncomment the cell below for Bin Interval
Note: Since there are so few instances of "Stage 3", its occurrences are dropped from the dataset so that they don't mess up the models

In [7]:
# #UNCOMMENT FOR BIN INTERVAL
# cleaned_df = merged_df.iloc[:, [0, 8, 10, 11, 15, 16, 17, 18, 19, 20, 28, 29, 30, 39, 46]]
# cat_cols = ["early_interval", "phylum", "class", "order", "family", "genus", "cc", "geoplate", "formation", "Bin Interval"]
# #fixing typos
# cleaned_df["Bin Interval"].replace("Jiangshangian ", "Jiangshanian", inplace=True)
# cleaned_df["Bin Interval"].replace("Piabian", "Paibian", inplace=True)
# #remove stage 3
# cleaned_df = cleaned_df[cleaned_df["Bin Interval"] != "Stage 3"]
# cleaned_df["Bin Interval"].value_counts()

Tremadocian     6698
Guzhangian      3997
Jiangshanian    3873
Drumian         3222
Wuliuan         2846
Stage 10        2099
Paibian         1796
Stage 4          316
Name: Bin Interval, dtype: int64

### Uncomment the cell below for the data used up until week 9

In [8]:
# #UNCOMMENT FOR ORIGINAL (used until week 9)
# cleaned_df = merged_df.iloc[:, [0, 5, 6, 8, 10, 11, 15, 16, 17, 18, 19, 20, 25, 28, 29, 30, 39, 42, 44, 45, 46]]
# cat_cols = ["accepted_name_x", "accepted_rank_x", "early_interval_x", "phylum", "class", "order", "family", "genus", "cc", "geogscale", "geoplate", "formation", "stratscale", "Common", "Skeletal Material", "Bin Interval"]

### Features that were removed in the updated dataset
- accepted_name (col 5): removed because it's an organizational feature that I didn't realize wasn't an informational feature
- accepted_rank (col 6): removed because it's an organizational feature that I didn't realize wasn't an informational feature 

### Features that were removed in the updated dataset but *could* still be useful and may be re-added
- geogscale (col 25): removed because it caused too many rows with nulls to be dropped; could still be valuable, but trims out a lot of data
- stratscale (col 42): removed because it caused too many rows with nulls to be dropped; could still be valuable, but trims out a lot of data
- Common (col 44): removed because it is just a label and therefore doesn't add much value; could still be valuable if different Bin Interval naming conventions also have different common name conventions. However, I don't think this is the case, so it could just confuse the model, because it is basically just the same thing as genus

In [9]:
num_cols = ["max_ma", "min_ma", "paleolng", "paleolat"]

In [10]:
df = cleaned_df.dropna()
df.set_index("occurrence_no", inplace=True)

rename_cols = {"early_interval_x": "early_interval", 
               "max_ma_x": "max_ma", 
               "min_ma_x": "min_ma"}
df.rename(mapper=rename_cols, axis=1, inplace=True)

#encoding categorical columns
#note: label_encoding is a dict of dicts, where each sub-dict has the label encodings as keys and the actual names as values
#for example, label_encoding["accepted_name_x"][0] returns the accepted_name_x that is encoded as 0
label_encoding = {}
le = preprocessing.LabelEncoder()

for col in cat_cols:
    le.fit(df[col])
    df[col] = le.transform(df[col])
    temp_encoding = dict(zip(le.classes_, le.transform(le.classes_)))
    label_encoding[col] = {v: k for k, v in temp_encoding.items()}
    df[col] = df[col].astype("int64")

#just making sure the numerical columns are all floats
for col in num_cols:
    df[col] = df[col].astype("float64")
    

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21393 entries, 1251 to 1587269
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   early_interval  21393 non-null  int64  
 1   max_ma          21393 non-null  float64
 2   min_ma          21393 non-null  float64
 3   phylum          21393 non-null  int64  
 4   class           21393 non-null  int64  
 5   order           21393 non-null  int64  
 6   family          21393 non-null  int64  
 7   genus           21393 non-null  int64  
 8   cc              21393 non-null  int64  
 9   paleolng        21393 non-null  float64
 10  paleolat        21393 non-null  float64
 11  geoplate        21393 non-null  int64  
 12  formation       21393 non-null  int64  
 13  Bin Interval    21393 non-null  int64  
dtypes: float64(4), int64(10)
memory usage: 2.4 MB


In [12]:
df.head(10)

,early_interval,max_ma,min_ma,phylum,class,order,family,genus,cc,paleolng,paleolat,geoplate,formation,Bin Interval
occurrence_no,,,,,,,,,,,,,,
1251,6,541.0,485.4,2,62,36,142,281,5,-86.74,30.34,3,426,5
1252,6,541.0,485.4,2,62,120,295,1231,5,-86.74,30.34,3,426,5
1254,6,541.0,485.4,3,43,100,321,1536,5,-86.74,30.34,3,426,5
1255,6,541.0,485.4,12,36,64,211,899,5,-86.74,30.34,3,426,5
1262,6,541.0,485.4,2,62,85,29,131,34,-139.72,29.26,0,59,7
1263,6,541.0,485.4,2,62,120,295,1382,34,-139.72,29.26,0,59,7
1264,6,541.0,485.4,3,29,68,287,1342,34,-139.72,29.26,0,59,7
1265,6,541.0,485.4,12,36,64,211,899,34,-139.72,29.26,0,59,7
1266,6,541.0,485.4,2,62,120,10,53,5,-122.43,39.01,0,458,5


In [13]:
df_train, df_test = train_test_split(df, stratify=df["Skeletal Material"])
y_train = df_train["Skeletal Material"]
y_test = df_test["Skeletal Material"]
df_train = df_train.drop(columns=["Skeletal Material"])
df_test = df_test.drop(columns=["Skeletal Material"])

In [14]:
y_train = y_train.values
y_test = y_test.values
X_train = df_train.values
X_test = df_test.values